In [2]:
import cv2
import numpy as np
import os

# Directory containing reference images
images_dir = "C:/Users/Amy Diaz/Documents/pyrobo/senalestransito"

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=100)#, scaleFactor=1.2, nlevels=8)

# Initialize camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to open the camera")
    exit()

while True:
    # Read frames
    ret, frame = cap.read()

    # Check if the frame is captured successfully
    if not ret:
        print("Error: Unable to capture frame")
        break

    # Convert the frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edges = cv2.Canny(frame_gray, 20, 60)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Copy the original image to draw contours
    image_contours = frame.copy()

    # Iterate over each contour
    for contour in contours:
        # Calculate epsilon: the approximation accuracy
        epsilon = 0.02 # * cv2.arcLength(contour, True)

        # Approximate the contour
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Draw the original contour (in green)
        cv2.drawContours(image_contours, [contour], -1, (0, 255, 0), 2)
        # Draw the approximated contour (in red)
        cv2.drawContours(image_contours, [approx], -1, (0, 0, 255), 2)

    # Detect ORB features in the frame
    keypoints_frame, descriptors_frame = orb.detectAndCompute(frame_gray, None)

    # Initialize variables to store the best match
    best_match_filename = None
    best_match_count = 0

    # Iterate through all files in the directory
    for filename in os.listdir(images_dir):
        # Load the image
        img = cv2.imread(os.path.join(images_dir, filename))

        # Check if the image is loaded successfully
        if img is not None:
            # Convert the image to grayscale
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect ORB features in the image
            keypoints_img, descriptors_img = orb.detectAndCompute(img_gray, None)

            # Create feature matcher
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

            # Match descriptors of both images
            if descriptors_frame is not None and descriptors_img is not None:
                matches = bf.match(descriptors_frame, descriptors_img)
                match_count = len(matches)

                # If the current image has more matches than the previous best match, update the best match
                if match_count > best_match_count:
                    best_match_filename = filename
                    best_match_count = match_count

    # Show the best matched image
    if best_match_filename is not None:
        print(f"Best match: {best_match_filename}, Matches: {best_match_count}")
        matched_image = cv2.imread(os.path.join(images_dir, best_match_filename))
        matched_image = cv2.resize(matched_image, (frame.shape[1], frame.shape[0]))  # Resize to match frame size
        combined_output = cv2.hconcat([image_contours, matched_image])  # Combine frame and matched image horizontally
        cv2.imshow('Matched Image', combined_output)
    else:
        print("No match found")
        cv2.imshow('Matched Image', image_contours)

    # Break the loop if Enter or Esc key is pressed
    keyboard = cv2.waitKey(1)
    if keyboard in (13, 27):
        break

# Release the camera and destroy all windows
cap.release()
cv2.destroyAllWindows()
